# Mass-Spring Systems: Springs vs Distance Constraints

Solving ODEs for mechanical systems with different constraint types.

The hardest part of the project is to include mass_spring. Depending on your operating system one of the these 3 will be functional for you (or not)

In [ ]:
# Windows
# add the directory of the binary of the compiler youre using
# then add the path to the mechsystem folder inside build
# looks like:
import sys, os
os.add_dll_directory(r"C:\msys64\ucrt64\bin")
sys.path.insert(0, r"C:\Users\fabia\Personal\Uni\Shared\scicomp\schaf3\build\mechsystem")

In [ ]:
# MacOS
import sys, os
from pathlib import Path

import importlib.util

p = (Path.cwd().parent / "build" / "mechsystem").resolve()
sys.path.insert(0, str(p))

In [ ]:
# alternative
import sys
sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

In [ ]:
from mass_spring import *
import numpy as np
from pythreejs import *
from time import sleep

## 1. Simple Pendulum with Spring Force

In [30]:
# Create pendulum suspended by spring
pend_spring = MassSpringSystem3d()
pend_spring.gravity = (0, 0, -9.81)

f1 = pend_spring.add(Fix((0, 0, 1.0)))
m1 = pend_spring.add(Mass(1.0, (0.5, 0, 0.0)))
pend_spring.add(Spring(1.0, 100.0, (f1, m1)))

print(f"Pendulum with spring created")

Pendulum with spring created


In [31]:
# Visualization
masses_sp = []
for m in pend_spring.masses:
    masses_sp.append(
        Mesh(SphereBufferGeometry(0.1, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos))

fixes_sp = []
for f in pend_spring.fixes:
    fixes_sp.append(
        Mesh(SphereBufferGeometry(0.08, 16, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos))

# Spring lines
springpos = []
for s in pend_spring.springs:
    pA = pend_spring[s.connectors[0]].pos
    pB = pend_spring[s.connectors[1]].pos
    springpos.append([pA, pB])

springgeo = LineSegmentsGeometry(positions=springpos)
springmat = LineMaterial(linewidth=3, color='yellow')
springs = LineSegments2(springgeo, springmat)

axes = AxesHelper(1)
camera = PerspectiveCamera(position=[2, 2, 1.5], aspect=800/600)
camera.position = [0, 3, 0]
camera.up = (0, 0, 1)
camera.lookAt((0, 0, 0))
key_light = DirectionalLight(position=[3, 3, 3])
ambient_light = AmbientLight()

scene_sp = Scene(children=[*masses_sp, *fixes_sp, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer_sp = Renderer(camera=camera, scene=scene_sp, controls=[controller],
                        width=800, height=600)

renderer_sp

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(0.0, 3.0, 0.0), projectionMatrix=(1.0, …

In [32]:
# Simulate spring pendulum
print("Simulating spring pendulum...")
for step in range(5000):
    pend_spring.simulate(0.002, 1)
    
    for m, mvis in zip(pend_spring.masses, masses_sp):
        mvis.position = m.pos
    
    springpos = []
    for s in pend_spring.springs:
        pA = pend_spring[s.connectors[0]].pos
        pB = pend_spring[s.connectors[1]].pos
        springpos.append([pA, pB])
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    
    if step % 500 == 0:
        print(f"Step {step}, t={step*0.001:.2f}s")
    sleep(0.001)

print("Spring pendulum complete!")

Simulating spring pendulum...
Step 0, t=0.00s
Step 500, t=0.50s
Step 1000, t=1.00s
Step 1500, t=1.50s
Step 2000, t=2.00s
Step 2500, t=2.50s
Step 3000, t=3.00s
Step 3500, t=3.50s
Step 4000, t=4.00s
Step 4500, t=4.50s
Spring pendulum complete!


## 2. Simple Pendulum with Distance Constraint

In [44]:
# Create pendulum with rigid constraint
pend_const = MassSpringSystem3d()
pend_const.gravity = (0, 0, -9.81)

f1 = pend_const.add(Fix((0, 0, 1)))
m1 = pend_const.add(Mass(1.0, (0.6, 0, 0.2)))
pend_const.add(DistanceConstraint(1.0, (f1, m1)))

print(f"Pendulum with distance constraint created")

Pendulum with distance constraint created


In [47]:
# Visualization
masses_c = []
for m in pend_const.masses:
    masses_c.append(
        Mesh(SphereBufferGeometry(0.1, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos))

fixes_c = []
for f in pend_const.fixes:
    fixes_c.append(
        Mesh(SphereBufferGeometry(0.08, 16, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos))

# Constraint lines
constpos = []
for c in pend_const.constraints:
    pA = pend_const[c.connectors[0]].pos
    pB = pend_const[c.connectors[1]].pos
    constpos.append([pA, pB])

constgeo = LineSegmentsGeometry(positions=constpos)
constmat = LineMaterial(linewidth=3, color='yellow')
constraints_c = LineSegments2(constgeo, constmat)

axes_c = AxesHelper(1)
camera_c = PerspectiveCamera(position=[2, 2, 1.5], aspect=800/600)
camera_c.position = [0, 3, 0]
camera_c.up = (0, 0, 1)
camera_c.lookAt((0, 0, 0))
key_light_c = DirectionalLight(position=[3, 3, 3])
ambient_light_c = AmbientLight()

scene_c = Scene(children=[*masses_c, *fixes_c, constraints_c, axes_c, camera_c, key_light_c, ambient_light_c])
controller_c = OrbitControls(controlling=camera_c)
renderer_c = Renderer(camera=camera_c, scene=scene_c, controls=[controller_c],
                       width=800, height=600)
renderer_c

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(0.0, 3.0, 0.0), projectionMatrix=(1.0, …

In [48]:
# Simulate constraint pendulum
print("Simulating constraint pendulum...")
for step in range(5000):
    pend_const.simulate(0.002, 10)
    
    for m, mvis in zip(pend_const.masses, masses_c):
        mvis.position = m.pos
    
    constpos = []
    for c in pend_const.constraints:
        pA = pend_const[c.connectors[0]].pos
        pB = pend_const[c.connectors[1]].pos
        constpos.append([pA, pB])
    constraints_c.geometry = LineSegmentsGeometry(positions=constpos)
    
    if step % 500 == 0:
        print(f"Step {step}, t={step*0.001:.2f}s")
    sleep(0.001)

print("Constraint pendulum complete!")

Simulating constraint pendulum...
Step 0, t=0.00s
Step 500, t=0.50s
Step 1000, t=1.00s
Step 1500, t=1.50s
Step 2000, t=2.00s
Step 2500, t=2.50s
Step 3000, t=3.00s
Step 3500, t=3.50s
Step 4000, t=4.00s
Step 4500, t=4.50s
Constraint pendulum complete!


## 3. Double Pendulum: Springs vs Constraints

In [2]:
# Double pendulum with springs
double_spring = MassSpringSystem3d()
double_spring.gravity = (0, 0, -9.81)

f1 = double_spring.add(Fix((0, 0, 1.5)))
m1 = double_spring.add(Mass(0.8, (0.4, 0, 0.8)))
m2 = double_spring.add(Mass(0.8, (0.4, 0, 0.1)))

double_spring.add(Spring(0.7, 80.0, (f1, m1)))
double_spring.add(Spring(0.7, 80.0, (m1, m2)))

print("Double pendulum with springs created")

Double pendulum with springs created


In [3]:
# Visualization
masses_ds = []
for m in double_spring.masses:
    masses_ds.append(
        Mesh(SphereBufferGeometry(0.1, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos))

fixes_ds = []
for f in double_spring.fixes:
    fixes_ds.append(
        Mesh(SphereBufferGeometry(0.08, 16, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos))

# Spring lines
springpos_ds = []
for s in double_spring.springs:
    pA = double_spring[s.connectors[0]].pos
    pB = double_spring[s.connectors[1]].pos
    springpos_ds.append([pA, pB])

springgeo_ds = LineSegmentsGeometry(positions=springpos_ds)
springmat_ds = LineMaterial(linewidth=3, color='yellow')
springs_ds = LineSegments2(springgeo_ds, springmat_ds)

axes_ds = AxesHelper(1)
camera_ds = PerspectiveCamera(position=[2, 2, 1.5], aspect=800/600)
camera_ds.position = [0, 5, 0]
camera_ds.up = (0, 0, 1)
camera_ds.lookAt((0, 0, 0))
key_light_ds = DirectionalLight(position=[3, 3, 3])
ambient_light_ds = AmbientLight()

scene_ds = Scene(children=[*masses_ds, *fixes_ds, springs_ds, axes_ds, camera_ds, key_light_ds, ambient_light_ds])
controller_ds = OrbitControls(controlling=camera_ds)
renderer_ds = Renderer(camera=camera_ds, scene=scene_ds, controls=[controller_ds],
                        width=800, height=600)
renderer_ds

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(0.0, 5.0, 0.0), projectionMatrix=(1.0, …

In [4]:
# Simulate double spring pendulum
print("Simulating double spring pendulum...")
for step in range(5000):
    double_spring.simulate(0.002, 10)
    
    for m, mvis in zip(double_spring.masses, masses_ds):
        mvis.position = m.pos
    
    springpos_ds = []
    for s in double_spring.springs:
        pA = double_spring[s.connectors[0]].pos
        pB = double_spring[s.connectors[1]].pos
        springpos_ds.append([pA, pB])
    springs_ds.geometry = LineSegmentsGeometry(positions=springpos_ds)
    
    if step % 500 == 0:
        print(f"Step {step}")
    sleep(0.001)

print("Double spring pendulum done!")

Simulating double spring pendulum...
Step 0
Step 500
Step 1000
Step 1500
Step 2000
Step 2500
Step 3000
Step 3500
Step 4000
Step 4500
Double spring pendulum done!


In [11]:
# Double pendulum with constraints
double_const = MassSpringSystem3d()
double_const.gravity = (0, 0, -9.81)

f1 = double_const.add(Fix((0, 0, 1.5)))
m1 = double_const.add(Mass(0.8, (0.6, 0, 0.7)))
m2 = double_const.add(Mass(0.8, (0.6, 0, -0.3)))

double_const.add(DistanceConstraint(1.0, (f1, m1)))
double_const.add(DistanceConstraint(1.0, (m1, m2)))

print("Double pendulum with constraints created")

Double pendulum with constraints created


In [12]:
# Visualization
masses_dc = []
for m in double_const.masses:
    masses_dc.append(
        Mesh(SphereBufferGeometry(0.1, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos))

fixes_dc = []
for f in double_const.fixes:
    fixes_dc.append(
        Mesh(SphereBufferGeometry(0.08, 16, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos))

# Constraint lines
constpos_dc = []
for c in double_const.constraints:
    pA = double_const[c.connectors[0]].pos
    pB = double_const[c.connectors[1]].pos
    constpos_dc.append([pA, pB])

constgeo_dc = LineSegmentsGeometry(positions=constpos_dc)
constmat_dc = LineMaterial(linewidth=3, color='yellow')
constraints_dc = LineSegments2(constgeo_dc, constmat_dc)

axes_dc = AxesHelper(1)
camera_dc = PerspectiveCamera(position=[2, 2, 1.5], aspect=800/600)
camera_dc.position = [0, 5, 0]
camera_dc.up = (0, 0, 1)
camera_dc.lookAt((0, 0, 0))
key_light_dc = DirectionalLight(position=[3, 3, 3])
ambient_light_dc = AmbientLight()

scene_dc = Scene(children=[*masses_dc, *fixes_dc, constraints_dc, axes_dc, camera_dc, key_light_dc, ambient_light_dc])
controller_dc = OrbitControls(controlling=camera_dc)
renderer_dc = Renderer(camera=camera_dc, scene=scene_dc, controls=[controller_dc],
                        width=800, height=600)
renderer_dc

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(0.0, 3.0, 0.0), projectionMatrix=(1.0, …

In [13]:
# Simulate double constraint pendulum
print("Simulating double constraint pendulum...")
for step in range(5000):
    double_const.simulate(0.002, 10)
    
    for m, mvis in zip(double_const.masses, masses_dc):
        mvis.position = m.pos
    
    constpos_dc = []
    for c in double_const.constraints:
        pA = double_const[c.connectors[0]].pos
        pB = double_const[c.connectors[1]].pos
        constpos_dc.append([pA, pB])
    constraints_dc.geometry = LineSegmentsGeometry(positions=constpos_dc)
    
    if step % 500 == 0:
        print(f"Step {step}")
    sleep(0.001)

print("Double constraint pendulum done!")

Simulating double constraint pendulum...
Step 0
Step 500
Step 1000
Step 1500
Step 2000
Step 2500
Step 3000
Step 3500
Step 4000
Step 4500
Double constraint pendulum done!


## 4. Spinning Top (Kreisel) with Precession

In [ ]:
# Create spinning top (Kreisel) with 3 masses connected by distance constraints
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# Create a fixed center point (the pivot)
center_fix = mss.add(Fix((0, 0, 1.0)))

# Create 3 masses arranged in an equilateral triangle in the xy-plane
# centered at height z=2, with center of mass at origin
mass_val = 1.0
radius = 0.5  # radius of triangle from center (reduced for stability)

# Three positions forming equilateral triangle
pos1 = (radius * np.cos(0*2*np.pi/3), radius * np.sin(0*2*np.pi/3), 1.0)
pos2 = (radius * np.cos(1*2*np.pi/3), radius * np.sin(1*2*np.pi/3), 1.0)
pos3 = (radius * np.cos(2*2*np.pi/3), radius * np.sin(2*2*np.pi/3), 1.0)

m1 = mss.add(Mass(mass_val, pos1))
m2 = mss.add(Mass(mass_val, pos2))
m3 = mss.add(Mass(mass_val, pos3))

# Connect the three masses with distance constraints (rigid triangle)
side_length = 2.0 * radius * np.sin(np.pi/3)  # side length of equilateral triangle
mss.add(DistanceConstraint(side_length, (m1, m2)))
mss.add(DistanceConstraint(side_length, (m2, m3)))


# --- plane circle definition ---
H = 1.0
center_fix2 = mss.add(Fix((0, 0, 1.0 - H)))
L2 = float(np.sqrt(radius**2 + H**2))

# --- constrain ALL THREE masses to the circle in plane z=2 ---
for mi in (m1, m2, m3):
    mss.add(DistanceConstraint(radius, (mi, center_fix)))
    mss.add(DistanceConstraint(L2, (mi, center_fix2)))

print(f"Spinning top created")

Spinning top created


In [34]:
# Set initial velocities for spinning motion
# Create angular velocity around z-axis (vertical axis)
omega = 10.0  # angular velocity (rad/s)


for i, m in enumerate(mss.masses):
    x, y, z = m.pos
    # Velocity tangent to circle in xy-plane
    vx = -omega * y
    vy = omega * x
    vz = 0.0
    # Set velocity directly on the mass
    m.vel = (vx, vy, vz)


In [37]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.15, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.1, 16, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

# Create edges connecting the masses (constraints)
constraint_positions = []
for c in mss.constraints:
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    constraint_positions.append([pA, pB])

constraint_geo = LineSegmentsGeometry(positions=constraint_positions)
constraint_mat = LineMaterial(linewidth=2, color='yellow')
constraints = LineSegments2(constraint_geo, constraint_mat)

# Create axes for reference
axes = AxesHelper(2)

# Scene setup
view_width = 800
view_height = 600

camera_k = PerspectiveCamera(position=[3, 3, 3], aspect=view_width/view_height)
camera_k.position = [1, 5, 2]
camera_k.up = (0, 0, 1)
camera_k.lookAt((0, 0, 0))
key_light = DirectionalLight(position=[5, 5, 5])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, constraints, axes, camera_k, key_light, ambient_light])
controller = OrbitControls(controlling=camera_k)
renderer = Renderer(camera=camera_k, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(1.0, 5.0, 2.0), projectionMatrix=(1.0, …

In [38]:
# Simulate the spinning top
# Watch as gravity causes precession due to angular momentum
from time import sleep

dt = 0.0001  # very small time step for stability with constraints
substeps = 100  # many solver iterations to help convergence

print("Starting simulation... watch the top precess!")
print(f"Time step: {dt} s, Solver steps: {substeps}")
print("This may take a while...\n")
#####
import numpy as np

dists = []
for c in mss.constraints:
    pA = np.array(mss[c.connectors[0]].pos, dtype=float)
    pB = np.array(mss[c.connectors[1]].pos, dtype=float)
    dists.append(np.linalg.norm(pA - pB))

print("Constraint distances:")
print("min / max:", min(dists), max(dists))
print(dists)
######
failed_steps = 0
for step in range(20000):
    try:
        # Simulate
        mss.simulate(0.001, 100)
    except ValueError as e:
        failed_steps += 1
        print(f"Step {step}: {e}")
        if failed_steps > 10:
            print("Too many convergence failures, stopping.")
            break
        continue
    
    # Update visualization
    for m, mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])
    
    # Update constraint edges
    constraint_positions = []
    for c in mss.constraints:
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        constraint_positions.append([pA, pB])
    
    constraints.geometry = LineSegmentsGeometry(positions=constraint_positions)
    
    # Print progress
    if step % 2000 == 0 and step > 0:
        print(f"Step {step}, t = {step*dt:.2f} s, Convergence failures: {failed_steps}")
    
    sleep(0.001)

print(f"Simulation complete! Total convergence failures: {failed_steps}")


Starting simulation... watch the top precess!
Time step: 0.0001 s, Solver steps: 100
This may take a while...

Constraint distances:
min / max: 0.49999999999999994 1.118033988749895
[np.float64(0.8660254037844386), np.float64(0.8660254037844386), np.float64(0.5), np.float64(1.118033988749895), np.float64(0.49999999999999994), np.float64(1.118033988749895), np.float64(0.5), np.float64(1.118033988749895)]
Step 2000, t = 0.20 s, Convergence failures: 0


KeyboardInterrupt: 